In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('/kaggle/input/restaurant-revenue-prediction/train.csv.zip')

In [ ]:
test = pd.read_csv('../input/restaurant-revenue-prediction/test.csv.zip')

In [ ]:
test

In [ ]:
df.describe().T

In [ ]:
df

In [ ]:
target = df['revenue']
train = df.drop('revenue', axis=1)

In [ ]:
features_train, features_valid, target_train, target_valid = train_test_split(train, target,
                                                                              test_size=0.2, random_state=12345)

In [ ]:
[features_train.columns.get_loc(c) for c in features_train.dtypes[features_train.dtypes == 'object'].index]

In [ ]:
cat_features=[features_train.columns.get_loc(c) for c in features_train.dtypes[features_train.dtypes == 'object'].index]

In [ ]:
params = {'iterations': 3000,
          'eval_metric':'RMSE',
          'depth': 16,
          'l2_leaf_reg': 3,
          'verbose': 100, 
          'learning_rate': 0.03,
          'od_type':'Iter',
          'od_wait':100
}
model = CatBoostRegressor(**params)
model.fit(features_train,target_train,
          cat_features=cat_features,
          eval_set=(features_valid, target_valid),
          plot=True
          )

In [ ]:
predicted_test = model.predict(test)

In [ ]:
predicted_test

In [ ]:
out = test['Id'].to_frame().copy()
out['Prediction'] = predicted_test

In [ ]:
out.to_csv('my_submission.csv', index=False)